In [1]:
!pip install pandas spacy
!python -m spacy download en_core_web_sm
!pip install gspread
!pip install oauth2client
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import gspread
from google.colab import auth
import pandas as pd
import math

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
# Load the Google Sheet
#sheet_url = 'https://docs.google.com/spreadsheets/d/15VfyipYKpd53oNU2A442paDCrqXebM6-WZaqVb3uFhc/edit#gid=0'
sheet_url = 'https://docs.google.com/spreadsheets/d/15VfyipYKpd53oNU2A442paDCrqXebM6-WZaqVb3uFhc/edit?usp=sharing'
spreadsheet = gc.open_by_url(sheet_url)


worksheet = spreadsheet.get_worksheet(1)

# Get all records from the sheet
records = worksheet.get_all_records()

# Convert records to DataFrame
ndf = pd.DataFrame(records)
df = ndf
ndf.head()

,Department,Job Title,Formatted common Traits,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,Accounting,Senior Tax and Accounting Specialist,"['communication', 'attention to detail', 'team...",-,High,High,High,Low
1,Accounting,Accounting Intern,[''],-,-,-,-,-
2,Accounting,Billing Manager,['communication'],-,-,High,High,Low
3,Accounting,Accounts Payable Specialist,"['time management', 'organizational skills', '...",Medium,High,High,High,Low
4,Accounting,Medical Biller,"['communication', 'problem-solving']",High,High,High,High,Low


In [11]:
import pandas as pd
from tabulate import tabulate

# # Sample dataset
# data = {
#     'Department': ['Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting'],
#     'Job Title': ['Senior Tax and Accounting Specialist', 'Accounting Intern', 'Billing Manager', 'Accounts Payable Specialist', 'Medical Biller'],
#     'Formatted common Traits': [['communication', 'attention to detail', 'team player'], [''], ['communication'], ['time management', 'organizational skills', 'communication'], ['communication', 'problem-solving']],
#     'Openness': ['-', '-', '-', 'Medium', 'High'],
#     'Conscientiousness': ['High', '-', '-', 'High', 'High'],
#     'Extraversion': ['High', '-', 'High', 'High', 'High'],
#     'Agreeableness': ['High', '-', 'High', 'High', 'High'],
#     'Neuroticism': ['Low', '-', 'Low', 'Low', 'Low']
# }

# df = pd.DataFrame(data)

# Person's traits
person_traits = {
    'Openness': 'Medium',
    'Conscientiousness': 'Medium',
    'Extraversion': 'High',
    'Agreeableness': 'High',
    'Neuroticism': 'Low'
}

def suggest_jobs(person_traits, df, min_trait_matches=3):
    suggestions = []
    for index, row in df.iterrows():
        matched_traits = 0
        matched_traits_list = []
        for trait in ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']:
            if row[trait] == person_traits[trait]:
                matched_traits += 1
                matched_traits_list.append(trait)
        if matched_traits >= min_trait_matches:
            suggestions.append({
                'Department': row['Department'],
                'Job Title': row['Job Title'],
                'Matched Traits': matched_traits,
                'Required Traits': matched_traits_list
            })
    return suggestions

suggested_jobs = suggest_jobs(person_traits, df)

# Convert suggestions to DataFrame for better display
suggested_jobs_df = pd.DataFrame(suggested_jobs)

# Print suggestions
if not suggested_jobs_df.empty:
    print("Suggested Jobs:")
    #print(suggested_jobs_df)
    print(tabulate(suggested_jobs_df, headers='keys', tablefmt='pretty'))
else:
    print("No job matches found with the given criteria.")


Suggested Jobs:
+-----+----------------------------+---------------------------------------------------+----------------+--------------------------------------------------------------+
|     |         Department         |                     Job Title                     | Matched Traits |                       Required Traits                        |
+-----+----------------------------+---------------------------------------------------+----------------+--------------------------------------------------------------+
|  0  |         Accounting         |       Senior Tax and Accounting Specialist        |       3        |       ['Extraversion', 'Agreeableness', 'Neuroticism']       |
|  1  |         Accounting         |                  Billing Manager                  |       3        |       ['Extraversion', 'Agreeableness', 'Neuroticism']       |
|  2  |         Accounting         |            Accounts Payable Specialist            |       4        | ['Openness', 'Extraversion', 'Agr

In [8]:
suggested_jobs_df

,Department,Job Title,Matched Traits,Required Traits
0,Accounting,Senior Tax and Accounting Specialist,3,"[Conscientiousness, Agreeableness, Neuroticism]"
1,Accounting,Accounts Payable Specialist,3,"[Conscientiousness, Agreeableness, Neuroticism]"
2,Accounting,Medical Biller,4,"[Openness, Conscientiousness, Agreeableness, N..."
3,Accounting,Cost analyst,4,"[Openness, Conscientiousness, Agreeableness, N..."
4,Accounting,Mortgage Loan Officer,4,"[Openness, Conscientiousness, Agreeableness, N..."
...,...,...,...,...
570,Retail,Retail Manager,4,"[Openness, Conscientiousness, Agreeableness, N..."
571,Retail,Area Manager,4,"[Openness, Conscientiousness, Agreeableness, N..."
572,Retail,Retail Buyer,3,"[Conscientiousness, Agreeableness, Neuroticism]"
573,Retail,Category Manager,4,"[Openness, Conscientiousness, Agreeableness, N..."
